Uso de un socket TCP

TCP es el acrónimo de Transmission Control Protocol, es decir, protocolo de control de transmisión, desarrollado en 1973 y documentado en la RFC 793. A continuación, se ha ubicado en el modelo OSI en el seno de la capa de transporte, y se ha extendido debido a su fiabilidad.

Se caracteriza por la manera de implementar la sincronización entre el cliente y el servidor y por la capacidad de dividir en segmentos de bytes la información que se ha de transmitir, siendo cada segmento perfectamente identificable y disponiendo de un sistema de control de integridad que hace que aquel que recibe un paquete pueda saber si el paquete se ha corrompido y volver a solicitarlo.

El flujo TCP utiliza los sockets, y es el módulo del mismo nombre de Python el que nos va a permitir trabajar con TCP.
La idea consiste en realizar un miniservidor de datos clave-valor muy básico, lo más sencillo posible, de cara a ver cómo crear un servidor y, a continuación, un cliente, pero también cómo manipular los datos que se intercambian entre sí.

Es imprescindible comprender que los datos que se transmiten desde un servidor hacia un cliente o desde un cliente hacia un servidor son
bytes y nada más. En Python 2, esto podría llevar a confusión, puesto que el tipo strde Python 2 era similar a los bytes. El tipo de
Python 3strrepresenta una cadena de caracteres en Unicode y el tipo bytes permite gestionar bytes, tal y como se explica en el capítulo
Tipos de datos y algoritmos aplicados.
Este punto es especialmente importante, pues la mayoría de los ejemplos presentes en la red, por otro lado de excelente calidad, se escriben
para Python 2 y generan cierta confusión sobre el tipo de datos que realmente envían.
Vamos a crear un servidor que reciba un número y que devuelva un código en función de si este número es primo o no.
He aquí el código de la función:


In [ ]:
def isprime(n):
  '''check ifintegernisaprime'''
#negativenumbers,0and1arenotprimes
  if n<2:
    return False
#2 istheonlyeven primenumber
  if n==2:
    return True
#allotherevennumbersarenotprimes
  if not n&1:
    return False
  #rangestarts with3andonlyneedstogoupthesquareroot
  #ofnforall oddnumbers
  for x in range(3,int(n**0.5)+1,2):
    if  n%x==0:
      return False
    return True

He aquí el código del servidor

In [ ]:
#!/usr/bin/python3
import socket
params=('127.0.0.1',8809)
BUFFER_SIZE=1024 #default
datos= {}
s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)#Internet,TCP
s.bind(params)
s.listen(1)

Se inicia el servidor y escucha en el puerto 8809, el método accept espera que se presente una conexión y, cuando llega, devuelve los
elementos necesarios para que la conexión pueda establecerse. Esto se realiza dentro de un bucle sin fin. El principio consiste en que,
mientras dure el intercambio entre el cliente y el servidor, se procese.


In [ ]:
conn,addr=s.accept()
print('Conexiónaceptada:%s'%str(addr))
while True:
	data =conn.recv(BUFFER_SIZE)
	if not data:
		break
	print('Datorecibido:%s'%data)
	try:number=int(data)
	except:
		response=b'E'
		phrase ="'%s'noesunnúmeroentero"%data
	else:
		if isprime(number):
			phrase= "’%s’esunnúmeroprimo"%number
			response =b'T'
		else:
			phrase= "’%s’noesunnúmeroprimo"%number
			response =b'F'
	finally:
		print(response)
		print(phrase)
		conn.send(response)
		conn.close()

La parte más compleja no es tanto la gestión de la conexión y de las transferencias de red como el propio procesamiento de los datos. Lo que
cuenta es saber qué hacer con los datos y cómo implementar un diálogo entre el cliente y el servidor, sabiendo que ambos elementos
intercambian bytes.
En este ejemplo, se sigue la siguiente convención: si no se comprende el dato transmitido por el cliente, se devuelve un código de error E. Si el
número es primo, se devuelve un código T (por True); en caso contrario, se devuelve un código F (por False).


Las adaptaciones realizadas en este ejemplo son muy básicas y limitadas: se busca emitir la menor cantidad de datos posible, pero el servidor
y el cliente deben entenderse para comprenderse entre sí: el cliente deberá conocer el conjunto de códigos susceptibles de ser enviados por
parte del servidor y gestionarlos.
Esto pone de manifiesto la problemática típica que se plantea cuando se busca manipular datos utilizando capas de bajo nivel.
He aquí el código del cliente, que se lee en contraposición con el del servidor:

In [ ]:
#!/usr/bin/python3
import socket
params=('127.0.0.1',8809)
BUFFER_SIZE=1024 #default
s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.connect(params)

Se envía una sucesión de mensajes de cara a cubrir todos los casos posibles de procesamiento del lado del servidor.

In [ ]:
cifras=[4j,4,5,-5,17,29,2**50,2**50-1]


Junto a cada mensaje, que en realidad es una especie de instrucción dada al servidor, se incluye un comentario sobre el comportamiento
esperado.
He aquí el código que realiza las consultas al servidor y que procesa las respuestas:


In [ ]:
for cifra in cifras:
	s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
	s.connect(params)
	print("Envíodeunmensaje%s"%cifra)
	s.send(('%s\n' %cifra).encode('utf8'))
	data=s.recv(BUFFER_SIZE)
	if len(data)==0:
		print("\tSinrespuesta")
	elif data==b'E\n':
		print("\tSehaproducidounerroren elservidor")
	elif data==b'T\n':
		print("\tSetratadeunnúmeroprimo")
	elif data==b'F\n':
		print("\tElnúmeronoesprimo")
	else:
		print("\tEldato recibidonosecomprende:%s" %data)
s.close()

mientras se trabaja dentro del bucle, se comunica con el servidor y, por tanto, el servidor se mantiene en el bucle infinito de
comunicación. El hecho de cerrar la conexión del lado del cliente envía una trama vacía que permite pasar al código break correspondiente del
lado del servidor (if not data: break) y, de este modo, liberarlo.
Si se realizara un bucle en torno al método accept, el servidor seguiría esperando nuevos datos o bien una conexión nueva, hasta el infinito,
pero en este caso no existe este bucle y finaliza.
Este ejemplo permite comunicarse con un único cliente y, a continuación, detener el servidor. Existen, en la práctica, casos de uso como este.

Creación de un servidor TCP

Como hemos visto antes, la gestión de un servidor TCP es una sucesión de etapas clásicas y reproducibles sea cual sea el tipo de servidor que
queramos construir. La parte más compleja consiste en determinar la manera de procesar los datos recibidos para realizar acciones y devolver
una respuesta adaptada.

Esto se ha modelado en el módulo socketserver(SocketServerpara la rama 2.x de Python), que ofrece dos clases asignadas a esta
problemática: TCPServer, que permite gestionar toda la problemática vinculada a la comunicación con el cliente,
yStreamRequestHandler, que permite gestionar la problemática de gestión de consultas, es decir, cómo procesar los datos que se
reciben.

De este modo, para reproducir el mismo trabajo que hemos visto en la sección anterior, el código del servidor sería algo así:

In [ ]:
#!/usr/bin/python3
import socketserver
params=('127.0.0.1',8809)
datos= {}
class PrimeTCPHandler(socketserver.StreamRequestHandler):
    def handle(self):
        data =self.rfile.readline().strip()
        try:
            number =int(data)
        except:
            response=b'E'
            phrase ="’%s’noesunnúmeroentero"%data
        else:
            if isprime(number):
                phrase= "’%s’esunnúmeroprimo"%number
                response =b'T'
            else:
                phrase= "’%s’noesunnúmeroprimo"%number
                response =b'F'
        finally:
            print(response)
        with open("prime_server.log","a") as f:
            f.write(phrase+'\n')
        return self.wfile.write(response +b'\n')
if __name__=='__main__':
    server= socketserver.TCPServer(params,PrimeTCPHandler)
    server.serve_forever()


Se pone de relieve la clase que permite procesar la consulta, bien separada de las dos simples líneas que gestionan el servidor TCP.
Esta separación facilita la accesibilidad del código, su legibilidad, y deja también las puertas abiertas a los diseñadores permitiéndoles realizar simplemente una pequeña arquitectura para gestionar de forma correcta sus handlers, que contienen el código de negocio.

Cabe destacar el uso del atributo rfilepara leer los datos que se reciben y wfilepara escribir aquellos que se envían. Destacaremos que
el conjunto de comunicación con el cliente se procesa en un handler, que incluye el establecimiento de la conexión, el intercambio de datos, la
respuesta y el final de la conexión.

Esto quiere decir que el cliente DEBE ser capaz de tener en cuenta estos cambios y establecer una nueva conexión con cada consulta enviada:

In [ ]:
!/usr/bin/python3
import socket
params=('127.0.0.1',8809)
BUFFER_SIZE=1024 #default
cifras=[4j,4,5,17,29,2**50,2**50-1]
for cifra in cifras:
    t0=time()
    s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s.connect(params)
    print("Envíodeunmensaje%s"%cifra)
    s.send(('%s\n' %cifra).encode('utf8'))
    data=s.recv(BUFFER_SIZE)
    if len(data)==0:
        print("\tSinrespuestas")
    elif data==b'E\n':
        print("\tSehaproducidounerroren elservidor")
    elif data==b'T\n':
        print("\tSetratadeunnúmeroprimo")
    elif data==b'F\n':
        print("\tElnúmeronoesprimo")
    else:
        print("\tEldato recibidonosecomprende:%s" %data)
    s.close()
    print("\tTiempoutilizado:%s"%(time()-t0))

En efecto, cada vez que se envía un nuevo mensaje se vuelve a crear el socket y se reinicia la conexión, lo cual puede representar un trabajo
suplementario.
Existen también clases más básicas que son más próximas a las de bajo nivel y que establecen un puente entre lo que hemos visto al principio
y estas dos últimas clases (http://docs.python.org/py3k/library/socketserver.html).
